<b>

ISYS613 | Course Project | Fall 2021
--------|----------------|----------

</b>


## Project Overview
The objective of this project is to afford you the opportunity to demonstrate the
skills acquired over the course of the semester.  Specifically, you will be constructing
an analytics-ready dataset using data from several disparate, but related data sources.

Your dataset is to be a pandas dataframe suitable for answering the proof-of-concept
questions located at the end of this document.

### The Scenario
You are the proprietor of a small tour company.  You are interested in launching a new app
that will allow users to undertake self-guided tours of noteworthy authors. To launch your app
you need to assemble the authors' location information along with some other facts that you
believe will be of interest to your customers.

## Data Sources
You will be collecting and assembling data from several different sources and formats.
1. Source 1 - web scraping
1. Source 2 - delimitted file
1. Source 3 - direct download XML file
1. Source 4 - Zip Code API

## Project Teams
Working on a team is not a requirement and you may complete the project on your own.
Thus, project team(s) may be composed of 1 or **at most** 2 students. If you are working
with another student please, ensure both of your names are clearly visible in your final solution JNB.


### Source 1: Initial Author Data
The initial list of author names is to be web-scraped from a popular "famous quotes" website.
There are 50 unique authors available from this site.  Begin your exploration at the base
URL and determine your own strategy to find and capture the required author data.

Depending on your approach, you may encounter the same author multiple times.
In the end, your list of authors should be duplicate free.

```
Base Scrape URL: https://quotes.toscrape.com
```
#### Tasks
1. Beginning at the Base Scrape URL given above, you are to use a web scraping technique to collect
the following author information:
* Full Name
* Date of birth
* Birth location
2. Create a pandas dataframe from (or using) the relevant data

In [1]:
from requests.exceptions import HTTPError
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# TEST DATA
URL = 'https://quotes.toscrape.com'

def simple_get(url, *args, **kwargs):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        resp = requests.get(url, *args, **kwargs)
        # If the response was successful, no Exception will be raised
        resp.raise_for_status()

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        raise http_err
    except Exception as err:
        print(f'Other error occurred: {err}')
        raise err

    return resp

all_links = [URL]

for i in all_links:
    response = simple_get(i, timeout = 5)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    next_l = soup.find('li', class_= 'next')
    if next_l is not None:
        next_a = next_l.a
        next_href = next_a.get('href')
        all_links.append(URL+next_href)

def author_link(soup_url):
    authors_dic = {}
    for link in soup_url.find_all('span'):
        if link.a != None:
            l = link.a
            url_info = l.get('href')
        if link.small != None:
            author = link.small
            name = author.text
            authors_dic[name] = URL+url_info
    return authors_dic


duplicate_free = []

for a in all_links:
    response_a = simple_get(a, timeout = 5)
    html_a = response_a.text
    soup_a = BeautifulSoup(html_a, 'html.parser')


    for key,val in author_link(soup_a).items():
        response_val = simple_get(val, timeout = 5)
        html_val = response_val.text
        soup_val = BeautifulSoup(html_val, 'html.parser')
    
        
        birthdate_tag = soup_val.find(class_='author-born-date')
        birthdate = birthdate_tag.text
        
        birthplace_tag = soup_val.find(class_='author-born-location')
        birthplace = birthplace_tag.text.strip(' in')
        
        
        
        info = [key,birthdate,birthplace]
        
        if info not in duplicate_free:
            duplicate_free.append(info)

source_1 = pd.DataFrame(duplicate_free, columns = ['Name','Birthdate','Birthplace']) 
source_1

,Name,Birthdate,Birthplace
0,Albert Einstein,"March 14, 1879","Ulm, Germany"
1,J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit..."
2,Jane Austen,"December 16, 1775","Steventon Rectory, Hampshire, The United Kingdom"
3,Marilyn Monroe,"June 01, 1926",The United States
4,André Gide,"November 22, 1869","Paris, France"
5,Thomas A. Edison,"February 11, 1847","Milan, Ohio, The United States"
6,Eleanor Roosevelt,"October 11, 1884",The United States
7,Steve Martin,"August 14, 1945","Waco, Texas, The United States"
8,Bob Marley,"February 06, 1945","Nine Mile, Saint Ann, Jamaica"
9,Dr. Seuss,"March 02, 1904","Springfield, MA, The United States"


In [2]:
source_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        50 non-null     object
 1   Birthdate   50 non-null     object
 2   Birthplace  50 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


### Source 2: Author Key Data
For each of the 50 authors previously identified, you are to merge a key and gender value available
from a CSV file with the author data from *Source 1*.

```
CSV File: *author_key_file.csv*
```
#### Tasks
1. The author names are unique in both
data sources and thus may be used to associate the *key* and *gender* attribute values with the author.
3. Once you have completed the merge, convert the *key* column to be the dataframe's row index.

In [3]:
import csv

with open('author_key_file.csv', mode='r', newline='', encoding = 'UTF-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    
    header = next(csv_reader)
    
    source = []
    for row in csv_reader: 
        name_handler = row[0]
        
        if name_handler == 'Alexandre Dumas-fils':
            fullname = name_handler.replace('Alexandre Dumas-fils','Alexandre Dumas fils')
        else:
            fullname = row[0]
        full_info = fullname,row[1],row[2]
        source.append(full_info)
    
source_2 = pd.DataFrame(source, columns = ['Name','Key','Gender']) 
merged_source = pd.merge(source_1,source_2, on='Name')

indexed_key = merged_source.set_index('Key')
indexed_key


,Name,Birthdate,Birthplace,Gender
Key,,,,
QWxiZXJ0,Albert Einstein,"March 14, 1879","Ulm, Germany",M
Si1LLVJv,J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",F
SmFuZS1B,Jane Austen,"December 16, 1775","Steventon Rectory, Hampshire, The United Kingdom",F
TWFyaWx5,Marilyn Monroe,"June 01, 1926",The United States,F
QW5kcmUt,André Gide,"November 22, 1869","Paris, France",M
VGhvbWFz,Thomas A. Edison,"February 11, 1847","Milan, Ohio, The United States",M
RWxlYW5v,Eleanor Roosevelt,"October 11, 1884",The United States,F
U3RldmUt,Steve Martin,"August 14, 1945","Waco, Texas, The United States",M
Qm9iLU1h,Bob Marley,"February 06, 1945","Nine Mile, Saint Ann, Jamaica",M


In [4]:
indexed_key.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, QWxiZXJ0 to TWFkZWxl
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        51 non-null     object
 1   Birthdate   51 non-null     object
 2   Birthplace  51 non-null     object
 3   Gender      51 non-null     object
dtypes: object(4)
memory usage: 2.0+ KB


### Source 3: Author Location Data
This direct download XML data source contains the location information that will be used
to direct app users to the historical sites associated with each author.  The *id*
tag can be used to match the previously assembled author data (ie, the *key* attribute) with
the author location data.

You are to retain all of the location attributes (excluding *id* - which will be a
duplicate of the existing *key* attribute) from this data source to extend the
previously collected author data.

```
Direct Download URL: https://www.drivehq.com/file/DFPublishFile.aspx/FileID7657515244/Keycqacws4cypvo/author_location_data.xml
Data Format: XML
Encoding: UTF-8
```
#### Tasks
1. Ingest the XML data into a pandas dataframe
2. Use the pandas dataframe merge method to join the new dataframe
with the Source 1+2 dataframe.
3. *Source 3* contains location data for many public figures in addition
to the 50 authors in which we are currently interested. You must restrict your
final Source 3 dataframe to our 50 authors.

In [5]:
import requests
from requests.exceptions import HTTPError
import xml.etree.ElementTree as ET
import pandas as pd
import io

URI = 'https://www.drivehq.com/file/DFPublishFile.aspx/FileID7657515244/Keycqacws4cypvo/author_location_data.xml'

def uri_get(url:str, *args, **kwargs) -> requests.Response:
    try:
        response = requests.get(url, *args, **kwargs)
        response.raise_for_status()

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        raise
    except Exception as err:
        print(f'Other error occurred: {err}')
        raise
    else:
        print(f'Success: {response.status_code}')

    return response


response = uri_get(URI)
root = ET.fromstring(response.text)

df_cols = ['Key','price','bedrooms','bathrooms','sqft_living','sqft_lot', 'floors','waterfront',
          'grade','yr_built','lat','long']
rows = []

for node in root:
    t_id_key = node.find('id').text
    t_price = node.find('price').text
    t_bedrooms = node.find('bedrooms').text
    t_bathrooms = node.find('bathrooms').text
    t_sqft_living = node.find('sqft_living').text
    t_sqft_lot = node.find('sqft_lot').text
    t_floors = node.find('floors').text
    t_waterfront = node.find('waterfront').text
    t_grade = node.find('grade').text
    t_yr_built = node.find('yr_built').text
    t_lat = node.find('lat').text
    t_long = node.find('long').text
    rows.append({'Key': t_id_key, 'price':t_price, 'bedrooms':t_bedrooms,
                 'bathrooms':t_bathrooms,'sqft_living':t_sqft_living,'sqft_lot': t_sqft_lot, 'floors':t_floors,
                 'waterfront':t_waterfront,'grade':t_grade,'yr_built':t_yr_built,
                 'lat':float(t_lat),'long':float(t_long)})
    
out_df = pd.DataFrame(rows,columns = df_cols)

merged_source123 = pd.merge(merged_source,out_df, on='Key')
merged_source123

    
    

Success: 200


,Name,Birthdate,Birthplace,Key,Gender,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,grade,yr_built,lat,long
0,Albert Einstein,"March 14, 1879","Ulm, Germany",QWxiZXJ0,M,699999,3,0.75,1240,4000,1,0,7,1968,40.7123,-94.0385
1,J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",Si1LLVJv,F,415000,3,2.5,1060,1536,2,0,8,2000,38.8778,-77.0672
2,Jane Austen,"December 16, 1775","Steventon Rectory, Hampshire, The United Kingdom",SmFuZS1B,F,300000,3,2.5,2540,5050,2,0,7,2006,33.9784,-117.0405
3,Marilyn Monroe,"June 01, 1926",The United States,TWFyaWx5,F,2480000,4,5,5310,16909,1,0,12,1992,42.8802,-78.8432
4,André Gide,"November 22, 1869","Paris, France",QW5kcmUt,M,425000,3,1.75,1530,9800,1,0,8,1958,37.6119,-82.7187
5,Thomas A. Edison,"February 11, 1847","Milan, Ohio, The United States",VGhvbWFz,M,179900,2,1,680,6400,1,0,6,1943,47.7890,-117.0142
6,Eleanor Roosevelt,"October 11, 1884",The United States,RWxlYW5v,F,420000,3,1.75,1770,6000,1,0,7,1952,34.6356,-102.7123
7,Steve Martin,"August 14, 1945","Waco, Texas, The United States",U3RldmUt,M,761000,3,3.5,2050,2020,2,0,8,2006,42.0902,-72.0532
8,Bob Marley,"February 06, 1945","Nine Mile, Saint Ann, Jamaica",Qm9iLU1h,M,307150,3,1.5,1480,6752,1,0,7,1959,32.7916,-96.7526
9,Dr. Seuss,"March 02, 1904","Springfield, MA, The United States",RHItU2V1,M,550000,3,3.5,2490,3582,2,0,8,2005,42.6669,-73.7866


In [6]:
merged_source123.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         51 non-null     object 
 1   Birthdate    51 non-null     object 
 2   Birthplace   51 non-null     object 
 3   Key          51 non-null     object 
 4   Gender       51 non-null     object 
 5   price        51 non-null     object 
 6   bedrooms     51 non-null     object 
 7   bathrooms    51 non-null     object 
 8   sqft_living  51 non-null     object 
 9   sqft_lot     51 non-null     object 
 10  floors       51 non-null     object 
 11  waterfront   51 non-null     object 
 12  grade        51 non-null     object 
 13  yr_built     51 non-null     object 
 14  lat          51 non-null     float64
 15  long         51 non-null     float64
dtypes: float64(2), object(14)
memory usage: 6.8+ KB


### Source 4: Zip Code API
You may have observed that the *Source 3* data does not include the city and zip code information for
the property locations for our 50 authors.  Your task is the use the latitude and longitude
values from the *Source 3* data to locate the city and zip code information and incorporate
that information into your dataframe.

The USPS maintains data related to all US zip codes and their centroid latitude and longitude.  The
*Source 3* data contains the latitude and longitude information (the lat and long tags) for the
author locations. Your task
is to use the *Source 3* latitude and longitude values as an argument to the zip code api in order to
to augment your dataframe with the city name and zip code of the authors' locations.

The ZIP code API contains ZIP codes for the
continental United States, Alaska, Hawaii, Puerto Rico, and American Samoa.
The API provides data in JSON format values for for ZIP code, city, latitude, longitude,
timezone (offset from GMT).

The relevant zip code API URL and parameters are shown in the table below.


API Information and Parameters | Value
---------------------|------
Dataset Name | US Zip Codes Points- United States of America
Dataset Overview | https://public.opendatasoft.com/explore/dataset/georef-united-states-of-america-zc-point/information/
API Documentation | https://public.opendatasoft.com/explore/dataset/georef-united-states-of-america-zc-point/api/
API URL | https://public.opendatasoft.com/api/records/1.0/search/
dataset (parameter) | georef-united-states-of-america-zc-point
rows (parameter) | 100
geofilter.distance (parameter) | Distance value must by at least 10 kilometers
Geofilter Documentation | https://help.opendatasoft.com/platform/en/exploring_catalog_and_datasets/03_searching_the_data/search.html#geo-filtering

Use the interactive OpenAPI interface available at the API Documentation URL to review the JSON data
returned by a properly constructed query.

#### Tasks
The long and lat values from *Source 3* are the exact longitude and latitude values for a
single property location.  As such, it is unlikely that you will find an exact match from the
zip code API using these values. Instead, you must:

1. round the lat and long values to 2 decimal place
2. invoke the API requesting all zip code related data within 10 kilometers from the
rounded lat and long values. (Refer to the *q* or *geofilter.distance* parameter documentation listed above.)
3. If the returned JSON results indicate more that one candidate record has been returned, you
must determine which record's longitude and latitude are the **closest** to the *Source 3*
long and lat values.
<br/><br/>There are several ways to accomplish this. For example, you could calculate the distance between the
each result's longitude,latitude values and the the long, lat values from *Source 3*. The
distance between the values can be calculated using the Pythagorean Theorem as follows: <br/>
&radic;( (lon<sub>1</sub>-lon<sub>2</sub>)<sup>2</sup> + (lat<sub>1</sub>-lat<sub>2</sub>)<sup>2</sup> )
 <br/><br/> Another, far better, approach involves a paying careful attention *dist* name,value pair
 located in the fields object of the returned JSON data and how that field may be used in the *sort*
 parameter and how its use impacts the returned results. This is a hint...
4. Use the city and zip code that are associated with the result having the
 minimum distance from (closest to) the *Source 3* long, lat values.


In [7]:
rounded_source = merged_source123.round({'lat':2,'long':2})
rounded_source

lat_source  = rounded_source['lat']
lon_source = rounded_source['long']
name = merged_source123['Name']


import requests
import json
from requests.exceptions import HTTPError
from pprint import pprint


def uri_get(url:str, *args, **kwargs) -> requests.Response:
    try:
        response = requests.get(url, *args, **kwargs)
        # If the response was successful, no Exception will be raised
        # by the following line
        # response.raise_for_status()

    except HTTPError as http_err:
        # Cannot proceed
        print(f'HTTP error occurred: {http_err}')
        raise
    except Exception as err:
        # Cannot proceed
        print(f'Other error occurred: {err}')
        raise
    else:
        response.status_code

    return response


URI = 'https://public.opendatasoft.com/api/records/1.0/search/'

results_zip = []
results_city = []

for (lat, lon) in zip(lat_source,lon_source):
    parameters = {'dataset':'georef-united-states-of-america-zc-point', 
                  'rows': 1,
                  'geofilter.distance': f'{lat},{lon},10000',
                  'sort': '-dist'}

    response = uri_get(URI, params=parameters,  timeout=5)
    py_dict = response.json() 

    for rec in py_dict['records']:
        fields = rec['fields']
        zcode = fields['zip_code']
        city = fields['usps_city'] 
        geo = rec['geometry']
        coordinates_lat = geo['coordinates'][1]
        coordinates_lon = geo['coordinates'][0]
        
        results_zip.append(zcode)
        results_city.append(city)
 

df_zip_city = pd.DataFrame({'Name':name, 'zip code': results_zip, 'city': results_city})


sources_final = pd.merge(rounded_source, df_zip_city, how = 'inner', on = 'Name')
sources_final

        

,Name,Birthdate,Birthplace,Key,Gender,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,grade,yr_built,lat,long,zip code,city
0,Albert Einstein,"March 14, 1879","Ulm, Germany",QWxiZXJ0,M,699999,3,0.75,1240,4000,1,0,7,1968,40.71,-94.04,50133,Kellerton
1,J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",Si1LLVJv,F,415000,3,2.5,1060,1536,2,0,8,2000,38.88,-77.07,22211,Fort Myer
2,Jane Austen,"December 16, 1775","Steventon Rectory, Hampshire, The United Kingdom",SmFuZS1B,F,300000,3,2.5,2540,5050,2,0,7,2006,33.98,-117.04,92320,Calimesa
3,Marilyn Monroe,"June 01, 1926",The United States,TWFyaWx5,F,2480000,4,5,5310,16909,1,0,12,1992,42.88,-78.84,14204,Buffalo
4,André Gide,"November 22, 1869","Paris, France",QW5kcmUt,M,425000,3,1.75,1530,9800,1,0,8,1958,37.61,-82.72,41601,Allen
5,Thomas A. Edison,"February 11, 1847","Milan, Ohio, The United States",VGhvbWFz,M,179900,2,1,680,6400,1,0,6,1943,47.79,-117.01,99025,Newman Lake
6,Eleanor Roosevelt,"October 11, 1884",The United States,RWxlYW5v,F,420000,3,1.75,1770,6000,1,0,7,1952,34.64,-102.71,79035,Friona
7,Steve Martin,"August 14, 1945","Waco, Texas, The United States",U3RldmUt,M,761000,3,3.5,2050,2020,2,0,8,2006,42.09,-72.05,1566,Sturbridge
8,Bob Marley,"February 06, 1945","Nine Mile, Saint Ann, Jamaica",Qm9iLU1h,M,307150,3,1.5,1480,6752,1,0,7,1959,32.79,-96.75,75223,Dallas
9,Dr. Seuss,"March 02, 1904","Springfield, MA, The United States",RHItU2V1,M,550000,3,3.5,2490,3582,2,0,8,2005,42.67,-73.79,12206,Albany


## Proof of Concept
Use the dataframe that you constructed above to answer the following three questions.

### Question 1
What is the mean and standard deviation of living space (sqft_living) by number of bedrooms? Use the pandas
dataframe.agg() method to calculate these statistics in a single step.

### Question 2
How many authors, by gender, live on lots that exceed the mean lot size (sqft_lot) by more
than 1 standard deviation.

### Question 3
Examine the documentation for the pandas dataframe.cut() method
[Docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html)

Use this method to convert the *price* attribute into to
3 approximately equal sized bins. Assign each bin specific labels of 'low', 'medium', 'high'.
How many authors are there in each price category?

In [8]:
import pandas as pd
import numpy as np


### First need to update the datatype for columns 5-12
merged_source123 = merged_source123.astype({"Name": object, "Birthdate": object, "Birthplace": object, "Key": object, "Gender": object, "price": float, "bedrooms": float, "bathrooms": float, "sqft_lot": float, "sqft_living": float, "floors": float, "waterfront": float, "grade": float, "yr_built": float})

###Question 1 
question1 = merged_source123.groupby('bedrooms')['sqft_living'].agg(['mean','std'])
print(question1)

###Question 2
val = merged_source123['sqft_lot'].mean() + merged_source123['sqft_lot'].std()
filter = merged_source123['sqft_lot'] > val
num_authors = merged_source123[filter]
question2 = num_authors.groupby('Gender')['Name'].count()
print()
print (question2)

###Question 3
question3 = pd.cut(merged_source123['price'], 3, labels = ["low", "medium", "high"])
merged_source123['price_1'] = question3
merged_source123

merged_source123.groupby('Name').count()
question_3 = merged_source123.groupby('price_1')['Name'].count()
print()
print (question_3)

                 mean          std
bedrooms                          
2.0        972.000000   270.314631
3.0       1838.888889   500.310160
4.0       2454.777778  1066.275130
5.0       4180.000000          NaN

Gender
F    1
M    2
Name: Name, dtype: int64

price_1
low       49
medium     1
high       1
Name: Name, dtype: int64


## Deliverable
Due to the the implementation of the JNB app, it is very easy to create circular variable dependencies. Such
dependencies will thwart my ability to run your entire JNB solution and result in the loss of valuable project points.

To ensure that your have not inadvertently created such dependencies, I **strongly** recommend you perform the following steps
prior to submitting your JNB.

1. Clear all outputs from your JNB solution's code cells
1. Save you JNB solution
1. Stop and restart your JNB kernel via the *Kernel* Jupyter Notebook menu item. Use the
*Restart & Run All Output* menu item.
1. Inspect your output cells for any errors

Once you have completed your project, upload the JNB containing your solution to the
*Course Project* assessment item located in the *Course Project* content area on our Blackboard site.